In [618]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [4]:
@memoized
def coef(n):
    return 1/(6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [8]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [96]:
def implies(a,b):
    # not a or b
    return (~a | b) & 1

In [7]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# f

In [135]:
def dh(n,a,b,c, regenerate=False):
    pos_c, pos_inds = h(n,x,y,z, regenerate)
    neg_c, neg_inds = h(n,y,x,z, regenerate)
    
    # Equivalent inequalities cancel out
    for p in pos_inds[:]:
        for i in neg_inds[:]:
            if equiv(p, i):
                pos_inds.remove(p)
                neg_inds.remove(i)
                break
                
#     # We now have r pos_inds and s neg_inds
#     # We generate all possible bit patterns (i.e. partitions) over all r+s indicators
    
#     rules = []  # 'rules' on possible bit patterns over reduced indicators
#     s = len(neg_inds)
#     for i, nind in enumerate(neg_inds):
#         for p, pind in enumerate(pos_inds):
#             b_nind = i   # neg_inds in lower bits (right half)
#             b_pind = p+s   # pos_inds in higher bits (left half) => sorting prioritizes making pos_inds 0
#             # generate rules for (r+s) bit integers
#             if not feasible(*nind, *pind, r=V):
#                 # nind => not pind = not nind or not pind = not(nind and pind)
#                 rules.append(lambda b: implies((b >> b_nind) & 1, ~((b >> b_pind) & 1)))
#             elif not non_redundant(*pind, r=V+nind):
#                 # nind => pind
#                 rules.append(lambda b: implies((b >> b_nind) & 1, (b >> b_pind) & 1))
#             elif not non_redundant(*nind, r=V+pind):
#                 # pind => nind
#                 rules.append(lambda b: implies((b >> b_pind) & 1, (b >> b_pind) & 1))
#             # the cases above are mutually exclusive, since we have no more nind <=> pind
#             # Later: not nind => pind, not pind => nind
    
    return (pos_c - neg_c), pos_inds, neg_inds

In [112]:
dh(3,x,y,z)

(0,
 [[7*x <= y],
  [5*x <= y],
  [6*x <= z],
  [5*x <= 3*y, y < 3*x],
  [3*x <= y + z, y < 3*x],
  [2*x <= y],
  [3*x <= y + z, z < 2*x],
  [6*x <= y],
  [5*x <= z, 2*x < y],
  [4*x <= y],
  [7*x <= z],
  [3*x <= y + z, y < 2*x],
  [x < -y + z],
  [3*x <= y + z, z < 3*x],
  [5*x <= 3*z, z < 3*x],
  [3*x <= 2*y],
  [2*x <= -y + z],
  [x < -y + z],
  [3*x <= -y + z],
  [3*y < z],
  [x <= -3*y + 3*z, -y + z < x],
  [x <= 3*y - z],
  [x <= -2*y + 2*z, z < 3*y]],
 [[7*y <= z],
  [2*y <= 3*x],
  [3*y - z <= x],
  [3*y - z <= x, z < 3*y],
  [5*y <= 3*z, z < 3*y],
  [3*y <= 5*x],
  [2*y - z <= 2*x, y < 3*x],
  [y <= 2*x],
  [3*y - z <= 3*x],
  [y <= 7*x, 3*x < y],
  [y <= 5*x, 2*x < z],
  [y + z <= 3*x],
  [x <= -3*y + z],
  [y <= 6*x, 2*x < y],
  [2*x < y],
  [y <= 4*x, 3*x < z],
  [y + z <= 3*x],
  [2*x <= -y + 2*z, z < 3*x]])

In [115]:
list(range(3,0,-1))

[3, 2, 1]

In [127]:
def count_ones(n):
    cnt = 0
    while n:
        n &= (n - 1)  # Remove the rightmost 1
        cnt += 1
    return cnt

In [150]:
def f(n, masks):
    tot = 0
    for i in range(n, 0, -1):
        tot += coef(i) * (hxyz[i][0] - hyxz[i][0] + count_ones(masks[(n<<1)-i]) - count_ones(masks[i-1]))
    return tot - thresh(n)

In [156]:
def extract(inds, mask):
    return [ind for i, ind in enumerate(inds) if (mask >> i) & 1]

In [157]:
extract([1,2,3,-1], 0b0110)

[2, 3]

In [162]:
def valid_region(n, rs, ss, hxyz, hyxz):
    region = concat(concat([extract(hxyz[i+1][1], rs[i]) for i in range(n)]) + concat([extract(hyxz[i+1][1], ss[i]) for i in range(n)]))
    print(region)
    return not (not feasible(*region, r=V))

In [160]:
n=3
hxyz = read(H_XYZ_CACHE, min_n=1, max_n=n)
hyxz = read(H_YXZ_CACHE, min_n=1, max_n=n)
h(n,y,x,z,False)

(0,
 [[7*y <= z],
  [2*y <= 3*x],
  [3*y - z <= x],
  [x <= -3*y + z],
  [3*y - z <= x, z < 3*y],
  [5*y <= 3*z, z < 3*y],
  [3*y <= 5*x],
  [2*y - z <= 2*x, y < 3*x],
  [y <= 2*x],
  [3*y - z <= 3*x],
  [y <= 7*x, 3*x < y],
  [3*x < y],
  [y <= 5*x, 2*x < z],
  [x <= -y + z],
  [y + z <= 3*x],
  [z < 2*x],
  [x <= -2*y + z, y < 2*x],
  [x <= -3*y + z],
  [y <= 6*x, 2*x < y],
  [2*x < y],
  [y <= 4*x, 3*x < z],
  [3*x <= -y + z],
  [-y + z < x],
  [3*x <= -y + 3*z, -y + z < x],
  [y + z <= 3*x],
  [2*x <= -y + 2*z, z < 3*x]])

In [165]:
len(hxyz[3][1]) + len(hxyz[2][1]) + len(hxyz[1][1])

36

In [170]:
len(hyxz[3][1]) + len(hyxz[2][1]) + len(hyxz[1][1])

30

In [172]:
count_ones((1 << 30) - 1)

30

In [173]:
f(3, [0,0,0], [(1 << len(hxyz[1][1])) - 1, (1 << len(hxyz[2][1])) - 1, (1 << len(hxyz[3][1])) - 1])

{1: (2, []), 2: (1, [[3*x <= y], [2*x <= z], [2*x <= y], [3*x <= z], [x <= -y + z]]), 3: (0, [[7*x <= y], [4*x <= z, 3*x < y], [5*x <= y], [6*x <= z], [5*x <= 3*y, y < 3*x], [3*x <= y + z, y < 3*x], [2*x <= y], [x <= -y + z], [3*x <= y + z, z < 2*x], [z < 2*x], [6*x <= y], [5*x <= z, 2*x < y], [4*x <= y], [7*x <= z], [y < 2*x], [3*x <= y + z, y < 2*x], [x < -y + z], [3*x <= -y + z], [3*x <= y + z, z < 3*x], [5*x <= 3*z, z < 3*x], [2*x <= z, -y + z < x], [3*x <= 2*y], [2*x <= -y + z], [x < -y + z], [3*x <= -y + z], [3*y < z], [x <= -3*y + z], [-y + z < x], [x <= -3*y + 3*z, -y + z < x], [x <= 3*y - z], [x <= -2*y + 2*z, z < 3*y]])}
{1: (1, []), 2: (1, [[3*y <= z], [y <= 3*x], [y <= 2*x], [x <= -y + z]]), 3: (0, [[7*y <= z], [2*y <= 3*x], [3*y - z <= x], [x <= -3*y + z], [3*y - z <= x, z < 3*y], [5*y <= 3*z, z < 3*y], [3*y <= 5*x], [2*y - z <= 2*x, y < 3*x], [y <= 2*x], [3*y - z <= 3*x], [y <= 7*x, 3*x < y], [3*x < y], [y <= 5*x, 2*x < z], [x <= -y + z], [y + z <= 3*x], [z < 2*x], [x <= 

-0.2157407407407408

In [174]:
valid_region(3, [0,0,0], [(1 << len(hxyz[1][1])) - 1, (1 << len(hxyz[2][1])) - 1, (1 << len(hxyz[3][1])) - 1], hxyz, hyxz)

[3*y <= z, y <= 3*x, y <= 2*x, x <= -y + z, 7*y <= z, 2*y <= 3*x, 3*y - z <= x, x <= -3*y + z, 3*y - z <= x, z < 3*y, 5*y <= 3*z, z < 3*y, 3*y <= 5*x, 2*y - z <= 2*x, y < 3*x, y <= 2*x, 3*y - z <= 3*x, y <= 7*x, 3*x < y, 3*x < y, y <= 5*x, 2*x < z, x <= -y + z, y + z <= 3*x, z < 2*x, x <= -2*y + z, y < 2*x, x <= -3*y + z, y <= 6*x, 2*x < y, 2*x < y, y <= 4*x, 3*x < z, 3*x <= -y + z, -y + z < x, 3*x <= -y + 3*z, -y + z < x, y + z <= 3*x, 2*x <= -y + 2*z, z < 3*x]


False

## @memoized
def all_inds(n,x,y,z):
    if n < 1:
        return H(n,x,y,z).indsall_inds(n-1,x,y,z)
    return 

def run(n):
    d = dh(n,x,y,z)
    checked = []
    for r in d.pos.inds:
        for s in d.neg.inds:
            fval = f(n, [r,s] + checked)
            print([r,s] + checked, fval)
            if fval <= thresh(n):
                print("Failed!")
                return False
        checked += [Not(r)]
    return True

def loop(n):
    while True:
        print(f'n = {n}')
        if run(n):
            print(n)
            break
        else:
            n += 1

In [210]:
from itertools import combinations, product
from collections import defaultdict
N=3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=N)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=N)
WS = [len(H_YXZ[i+1][1]) for i in range(N)] + [len(H_XYZ[i][1]) for i in range(N,0,-1)]

In [182]:
def f(n, counts):
    tot = 0
    for i in range(n, 0, -1):
        tot += coef(i) * (H_XYZ[i][0] - H_YXZ[i][0] + counts[(n<<1)-i] - counts[i-1])
    return tot - thresh(n)

In [192]:
def to_region(n, masks):
    return concat(concat([extract(H_XYZ[i+1][1], masks[(n<<1)-i-1]) for i in range(n)]) + concat([extract(H_YXZ[i+1][1], masks[i]) for i in range(n)]))

In [193]:
def valid_region(n, masks):
    return not (not feasible(*(to_region(n, masks)), r=V))

In [219]:
from itertools import combinations, product

def candidate_counts(n, ws):
    ''' ws = list of size 2n, holding the number of bits in each entry = max counts of 1s
    '''
    return [c for c in product(*(range(m+1) for m in ws)) if f(n,c) <= 0]
    
def sort_counts(counts, n):
    # maximize ones in the first half and minimize in the second half
    return sorted(counts, key=lambda count: (-sum(count[:n]), sum(count[n:])))

def candidate_masks(counts, ws, n):
    def gen_masks(count, w):
        return [sum(1 << p for p in pos) for pos in combinations(range(w), count)]  # possible bits to switch on

    return product(*[gen_masks(counts[i], ws[i]) for i in range(n<<1)])

def search(n):
    valid_counts = sort_counts(candidate_counts(n, WS), n)
    
    for c in valid_counts:
        print(c)
        ms = candidate_masks(c, WS, n)
        for m in ms:
            print(m)
            print(*['{:03b}'.format(i) for i in m])
            print()
            if valid_region(n,m):
                print("Failed! Region:")
                print(to_region(n, c))
                return True
            
    return False

In [220]:
search(3)

(0, 4, 26, 0, 0, 0)
(0, 15, 67108863, 0, 0, 0)
000 1111 11111111111111111111111111 000 000 000

(0, 4, 26, 0, 1, 0)
(0, 15, 67108863, 0, 1, 0)
000 1111 11111111111111111111111111 000 001 000

(0, 15, 67108863, 0, 2, 0)
000 1111 11111111111111111111111111 000 010 000

(0, 15, 67108863, 0, 4, 0)
000 1111 11111111111111111111111111 000 100 000

(0, 15, 67108863, 0, 8, 0)
000 1111 11111111111111111111111111 000 1000 000

(0, 15, 67108863, 0, 16, 0)
000 1111 11111111111111111111111111 000 10000 000

(0, 4, 26, 1, 0, 0)
(0, 15, 67108863, 1, 0, 0)
000 1111 11111111111111111111111111 001 000 000

(0, 15, 67108863, 2, 0, 0)
000 1111 11111111111111111111111111 010 000 000

(0, 15, 67108863, 4, 0, 0)
000 1111 11111111111111111111111111 100 000 000

(0, 15, 67108863, 8, 0, 0)
000 1111 11111111111111111111111111 1000 000 000

(0, 15, 67108863, 16, 0, 0)
000 1111 11111111111111111111111111 10000 000 000

(0, 15, 67108863, 32, 0, 0)
000 1111 11111111111111111111111111 100000 000 000

(0, 15, 67108863

(0, 15, 67108863, 1024, 2, 0)
000 1111 11111111111111111111111111 10000000000 010 000

(0, 15, 67108863, 1024, 4, 0)
000 1111 11111111111111111111111111 10000000000 100 000

(0, 15, 67108863, 1024, 8, 0)
000 1111 11111111111111111111111111 10000000000 1000 000

(0, 15, 67108863, 1024, 16, 0)
000 1111 11111111111111111111111111 10000000000 10000 000

(0, 15, 67108863, 2048, 1, 0)
000 1111 11111111111111111111111111 100000000000 001 000

(0, 15, 67108863, 2048, 2, 0)
000 1111 11111111111111111111111111 100000000000 010 000

(0, 15, 67108863, 2048, 4, 0)
000 1111 11111111111111111111111111 100000000000 100 000

(0, 15, 67108863, 2048, 8, 0)
000 1111 11111111111111111111111111 100000000000 1000 000

(0, 15, 67108863, 2048, 16, 0)
000 1111 11111111111111111111111111 100000000000 10000 000

(0, 15, 67108863, 4096, 1, 0)
000 1111 11111111111111111111111111 1000000000000 001 000

(0, 15, 67108863, 4096, 2, 0)
000 1111 11111111111111111111111111 1000000000000 010 000

(0, 15, 67108863, 4096, 4,

(0, 15, 67108863, 134217728, 1, 0)
000 1111 11111111111111111111111111 1000000000000000000000000000 001 000

(0, 15, 67108863, 134217728, 2, 0)
000 1111 11111111111111111111111111 1000000000000000000000000000 010 000

(0, 15, 67108863, 134217728, 4, 0)
000 1111 11111111111111111111111111 1000000000000000000000000000 100 000

(0, 15, 67108863, 134217728, 8, 0)
000 1111 11111111111111111111111111 1000000000000000000000000000 1000 000

(0, 15, 67108863, 134217728, 16, 0)
000 1111 11111111111111111111111111 1000000000000000000000000000 10000 000

(0, 15, 67108863, 268435456, 1, 0)
000 1111 11111111111111111111111111 10000000000000000000000000000 001 000

(0, 15, 67108863, 268435456, 2, 0)
000 1111 11111111111111111111111111 10000000000000000000000000000 010 000

(0, 15, 67108863, 268435456, 4, 0)
000 1111 11111111111111111111111111 10000000000000000000000000000 100 000

(0, 15, 67108863, 268435456, 8, 0)
000 1111 11111111111111111111111111 10000000000000000000000000000 1000 000

(0, 15, 67

(0, 15, 67108863, 516, 0, 0)
000 1111 11111111111111111111111111 1000000100 000 000

(0, 15, 67108863, 1028, 0, 0)
000 1111 11111111111111111111111111 10000000100 000 000

(0, 15, 67108863, 2052, 0, 0)
000 1111 11111111111111111111111111 100000000100 000 000

(0, 15, 67108863, 4100, 0, 0)
000 1111 11111111111111111111111111 1000000000100 000 000

(0, 15, 67108863, 8196, 0, 0)
000 1111 11111111111111111111111111 10000000000100 000 000

(0, 15, 67108863, 16388, 0, 0)
000 1111 11111111111111111111111111 100000000000100 000 000

(0, 15, 67108863, 32772, 0, 0)
000 1111 11111111111111111111111111 1000000000000100 000 000

(0, 15, 67108863, 65540, 0, 0)
000 1111 11111111111111111111111111 10000000000000100 000 000

(0, 15, 67108863, 131076, 0, 0)
000 1111 11111111111111111111111111 100000000000000100 000 000

(0, 15, 67108863, 262148, 0, 0)
000 1111 11111111111111111111111111 1000000000000000100 000 000

(0, 15, 67108863, 524292, 0, 0)
000 1111 11111111111111111111111111 10000000000000000100 

(0, 15, 67108863, 131104, 0, 0)
000 1111 11111111111111111111111111 100000000000100000 000 000

(0, 15, 67108863, 262176, 0, 0)
000 1111 11111111111111111111111111 1000000000000100000 000 000

(0, 15, 67108863, 524320, 0, 0)
000 1111 11111111111111111111111111 10000000000000100000 000 000

(0, 15, 67108863, 1048608, 0, 0)
000 1111 11111111111111111111111111 100000000000000100000 000 000

(0, 15, 67108863, 2097184, 0, 0)
000 1111 11111111111111111111111111 1000000000000000100000 000 000

(0, 15, 67108863, 4194336, 0, 0)
000 1111 11111111111111111111111111 10000000000000000100000 000 000

(0, 15, 67108863, 8388640, 0, 0)
000 1111 11111111111111111111111111 100000000000000000100000 000 000

(0, 15, 67108863, 16777248, 0, 0)
000 1111 11111111111111111111111111 1000000000000000000100000 000 000

(0, 15, 67108863, 33554464, 0, 0)
000 1111 11111111111111111111111111 10000000000000000000100000 000 000

(0, 15, 67108863, 67108896, 0, 0)
000 1111 11111111111111111111111111 1000000000000000000001

...

(0, 15, 67108863, 2099200, 9, 0)
000 1111 11111111111111111111111111 1000000000100000000000 1001 000

(0, 15, 67108863, 2099200, 17, 0)
000 1111 11111111111111111111111111 1000000000100000000000 10001 000

(0, 15, 67108863, 2099200, 6, 0)
000 1111 11111111111111111111111111 1000000000100000000000 110 000

(0, 15, 67108863, 2099200, 10, 0)
000 1111 11111111111111111111111111 1000000000100000000000 1010 000

(0, 15, 67108863, 2099200, 18, 0)
000 1111 11111111111111111111111111 1000000000100000000000 10010 000

(0, 15, 67108863, 2099200, 12, 0)
000 1111 11111111111111111111111111 1000000000100000000000 1100 000

(0, 15, 67108863, 2099200, 20, 0)
000 1111 11111111111111111111111111 1000000000100000000000 10100 000

(0, 15, 67108863, 2099200, 24, 0)
000 1111 11111111111111111111111111 1000000000100000000000 11000 000

(0, 15, 67108863, 4196352, 3, 0)
000 1111 11111111111111111111111111 10000000000100000000000 011 000

(0, 15, 67108863, 4196352, 5, 0)
000 1111 11111111111111111111111111 1000

(0, 15, 67108863, 536872960, 6, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 110 000

(0, 15, 67108863, 536872960, 10, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 1010 000

(0, 15, 67108863, 536872960, 18, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 10010 000

(0, 15, 67108863, 536872960, 12, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 1100 000

(0, 15, 67108863, 536872960, 20, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 10100 000

(0, 15, 67108863, 536872960, 24, 0)
000 1111 11111111111111111111111111 100000000000000000100000000000 11000 000

(0, 15, 67108863, 1073743872, 3, 0)
000 1111 11111111111111111111111111 1000000000000000000100000000000 011 000

(0, 15, 67108863, 1073743872, 5, 0)
000 1111 11111111111111111111111111 1000000000000000000100000000000 101 000

(0, 15, 67108863, 1073743872, 9, 0)
000 1111 11111111111111111111111111 10000000000000000001000

(0, 15, 67108863, 528384, 12, 0)
000 1111 11111111111111111111111111 10000001000000000000 1100 000

(0, 15, 67108863, 528384, 20, 0)
000 1111 11111111111111111111111111 10000001000000000000 10100 000

(0, 15, 67108863, 528384, 24, 0)
000 1111 11111111111111111111111111 10000001000000000000 11000 000

(0, 15, 67108863, 1052672, 3, 0)
000 1111 11111111111111111111111111 100000001000000000000 011 000

(0, 15, 67108863, 1052672, 5, 0)
000 1111 11111111111111111111111111 100000001000000000000 101 000

(0, 15, 67108863, 1052672, 9, 0)
000 1111 11111111111111111111111111 100000001000000000000 1001 000

(0, 15, 67108863, 1052672, 17, 0)
000 1111 11111111111111111111111111 100000001000000000000 10001 000

(0, 15, 67108863, 1052672, 6, 0)
000 1111 11111111111111111111111111 100000001000000000000 110 000

(0, 15, 67108863, 1052672, 10, 0)
000 1111 11111111111111111111111111 100000001000000000000 1010 000

(0, 15, 67108863, 1052672, 18, 0)
000 1111 11111111111111111111111111 100000001000000000000 

(0, 15, 67108863, 134221824, 12, 0)
000 1111 11111111111111111111111111 1000000000000001000000000000 1100 000

(0, 15, 67108863, 134221824, 20, 0)
000 1111 11111111111111111111111111 1000000000000001000000000000 10100 000

(0, 15, 67108863, 134221824, 24, 0)
000 1111 11111111111111111111111111 1000000000000001000000000000 11000 000

(0, 15, 67108863, 268439552, 3, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 011 000

(0, 15, 67108863, 268439552, 5, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 101 000

(0, 15, 67108863, 268439552, 9, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 1001 000

(0, 15, 67108863, 268439552, 17, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 10001 000

(0, 15, 67108863, 268439552, 6, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 110 000

(0, 15, 67108863, 268439552, 10, 0)
000 1111 11111111111111111111111111 10000000000000001000000000000 1010 000


KeyboardInterrupt: 

In [263]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [295]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
pos_ws = [len(H_XYZ[i][1]) for i in range(1,n+1)]
neg_ws = [len(H_YXZ[i][1]) for i in range(1,n+1)]
n_pos = sum(pos_ws)
n_neg = sum(neg_ws)
pos_coefs = [coef(i) for i in range(1,n+1)]
neg_coefs = [-c for c in pos_coefs]
pos_cs = concat([repeat(j, k) for j,k in zip(pos_coefs, pos_ws)])
neg_cs = concat([repeat(j, k) for j,k in zip(neg_coefs, neg_ws)])
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
const

0.16666666666666666

In [296]:
model = gp.Model()
model.Params.LazyConstraints = 1
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
model.Params.MIPFocus = 1
model.Params.Heuristics = 1
model.Params.CliqueCuts = 1
model.Params.FeasibilityTol = 1e-9
model.Params.IntFeasTol = 1e-9
model.Params.MarkowitzTol = 1e-4
model.Params.NodeLimit = 10000
pos_inds = model.addVars(n_pos, vtype=GRB.BINARY, name='x')
neg_inds = model.addVars(n_neg, vtype=GRB.BINARY, name='y')

Set parameter LazyConstraints to value 1
Set parameter MIPFocus to value 1
Set parameter Heuristics to value 1
Set parameter CliqueCuts to value 1
Set parameter FeasibilityTol to value 1e-09
Set parameter IntFeasTol to value 1e-09
Set parameter MarkowitzTol to value 0.0001
Set parameter NodeLimit to value 10000


In [279]:
pos_indicators = concat([H_XYZ[i][1] for i in range(1,n+1)])
neg_indicators = concat([H_YXZ[i][1] for i in range(1,n+1)])
for p, pi in enumerate(pos_indicators):
    for j, ni in enumerate(neg_indicators):
        if not feasible(*pi, *ni, r=V):
            # cannot both be true
            model.addConstr(pos_inds[p] + neg_inds[j] <= 1)
#         elif not non_redundant(*pi, r=V+ni) and not non_redundant(*ni, r=V+pi):
#             rules.append(pos_inds)

In [297]:
noise = gp.quicksum([neg_inds[i] for i in range(n_neg)])
noise = gp.quicksum([neg_inds[i] for i in range(n_neg)]) - \
        gp.quicksum([pos_inds[i] for i in range(n_pos)])
obj = gp.quicksum([pos_cs[i]*pos_inds[i] for i in range(n_pos)]) + \
        gp.quicksum([neg_cs[i]*neg_inds[i] for i in range(n_neg)]) + const

In [298]:
model.setObjective(obj+0.01*noise, GRB.MINIMIZE)
model.addConstr(obj <= thresh(n))

<gurobi.Constr *Awaiting Model Update*>

In [299]:
def to_region(n, selected_pos_inds, selected_neg_inds):
    all_pos_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
    all_neg_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
    on = concat([all_pos_inds[i] for i in selected_pos_inds] + [all_neg_inds[i] for i in selected_neg_inds])
    off = [all_pos_inds[i] for i in range(n_pos) if i not in selected_pos_inds] + [all_neg_inds[i] for i in range(n_neg) if i not in selected_neg_inds]
    off = [Not(i[0]) for i in off]  # Not(A and B) = Not A or Not B, we just take one
    return on + off

In [300]:
def valid_region(n, selected_pos_inds, selected_neg_inds):
    return not (not feasible(*to_region(n, selected_pos_inds, selected_neg_inds), r=V))

In [301]:
def valid_region_callback(model, where):
    if where == GRB.Callback.MIPSOL:
        try:
            pos_sol = model.cbGetSolution(pos_inds)
            neg_sol = model.cbGetSolution(neg_inds)
            selected_pos_inds = [i for i in pos_sol if pos_sol[i] > 0]
            selected_neg_inds = [i for i in neg_sol if neg_sol[i] > 0]
            # Check if the selected indicators form a valid region in the 3-simplex
            # Implement a function is_valid_region that checks validity
            if not valid_region(n, selected_pos_inds, selected_neg_inds):
#                 print("invalid region!!")
#                 print(to_region(n, selected_pos_inds, selected_neg_inds))
#                 print()
                
                # Add a constraint to the model to cut off this solution
                model.cbLazy(gp.quicksum([pos_inds[i] for i in selected_pos_inds]) \
                             + gp.quicksum([neg_inds[i] for i in selected_neg_inds]) \
                             <= len(selected_pos_inds) + len(selected_neg_inds) - 1)

        except gp.GurobiError as e:
            print(f"Gurobi error during callback: {e}")
        except Exception as e:
            print(f"Unexpected error during callback: {e}")

In [302]:
model.optimize(valid_region_callback)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 66 columns and 66 nonzeros
Model fingerprint: 0x6f7b878d
Variable types: 0 continuous, 66 integer (66 binary)
Coefficient statistics:
  Matrix range     [5e-03, 3e-02]
  Objective range  [5e-03, 2e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e-02, 7e-02]
Presolve time: 0.00s
Presolved: 1 rows, 66 columns, 66 nonzeros
Variable types: 0 continuous, 66 integer (66 binary)

Root relaxation: objective 9.722222e-02, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.09722    0    -          -    0.09722      -     -    0s
     0     0    0.09722    0    -          -    0.09722      -     -    0s
     0

In [303]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}, {not (not v.x)}")
    print(f"Objective value: {model.objVal}")
else:
    print("No feasible solution found.")

Optimal solution found:
x[0]: 0.0, False
x[1]: 0.0, False
x[2]: 0.0, False
x[3]: 0.0, False
x[4]: 0.0, False
x[5]: 0.0, False
x[6]: 0.0, False
x[7]: 1.0, True
x[8]: 0.0, False
x[9]: 0.0, False
x[10]: 0.0, False
x[11]: 1.0, True
x[12]: 0.0, False
x[13]: 0.0, False
x[14]: 0.0, False
x[15]: 0.0, False
x[16]: 0.0, False
x[17]: 0.0, False
x[18]: 0.0, False
x[19]: 0.0, False
x[20]: 0.0, False
x[21]: 0.0, False
x[22]: 0.0, False
x[23]: 0.0, False
x[24]: 0.0, False
x[25]: 0.0, False
x[26]: 0.0, False
x[27]: 0.0, False
x[28]: 0.0, False
x[29]: 0.0, False
x[30]: 0.0, False
x[31]: 0.0, False
x[32]: 0.0, False
x[33]: 0.0, False
x[34]: 0.0, False
x[35]: 1.0, True
y[0]: 1.0, True
y[1]: 1.0, True
y[2]: 1.0, True
y[3]: 0.0, False
y[4]: 0.0, False
y[5]: 0.0, False
y[6]: 0.0, False
y[7]: 0.0, False
y[8]: 0.0, False
y[9]: 0.0, False
y[10]: 0.0, False
y[11]: 0.0, False
y[12]: 0.0, False
y[13]: 0.0, False
y[14]: 0.0, False
y[15]: 0.0, False
y[16]: 0.0, False
y[17]: 0.0, False
y[18]: 0.0, False
y[19]: 0.0, 

In [304]:
pos_sol = [i for i,v in enumerate(model.getVars()) if v.x and v.varName[0] == 'x']
neg_sol = [i-n_pos for i,v in enumerate(model.getVars()) if v.x and v.varName[0] == 'y']
neg_sol

[0, 1, 2]

In [305]:
to_region(n, pos_sol, neg_sol)

[5*x <= y,
 2*x <= y,
 x <= -2*y + 2*z,
 z < 3*y,
 3*y <= z,
 y <= 3*x,
 y <= 2*x,
 3*x > y,
 2*x > z,
 2*x > y,
 3*x > z,
 x > -y + z,
 7*x > y,
 4*x > z,
 6*x > z,
 5*x > 3*y,
 3*x > y + z,
 x > -y + z,
 3*x > y + z,
 z >= 2*x,
 6*x > y,
 5*x > z,
 4*x > y,
 7*x > z,
 y >= 2*x,
 3*x > y + z,
 x >= -y + z,
 3*x > -y + z,
 3*x > y + z,
 5*x > 3*z,
 2*x > z,
 3*x > 2*y,
 2*x > -y + z,
 x >= -y + z,
 3*x > -y + z,
 3*y >= z,
 x > -3*y + z,
 -y + z >= x,
 x > -3*y + 3*z,
 x > 3*y - z,
 x > -y + z,
 7*y > z,
 2*y > 3*x,
 3*y - z > x,
 x > -3*y + z,
 3*y - z > x,
 5*y > 3*z,
 3*y > 5*x,
 2*y - z > 2*x,
 y > 2*x,
 3*y - z > 3*x,
 y > 7*x,
 3*x >= y,
 y > 5*x,
 x > -y + z,
 y + z > 3*x,
 z >= 2*x,
 x > -2*y + z,
 x > -3*y + z,
 y > 6*x,
 2*x >= y,
 y > 4*x,
 3*x > -y + z,
 -y + z >= x,
 3*x > -y + 3*z,
 y + z > 3*x,
 2*x > -y + 2*z]

In [306]:
feasible(*to_region(n, pos_sol, neg_sol), r=V)

False

In [307]:
gp.quicksum([pos_cs[i]*pos_inds[i].x for i in range(n_pos)]) + \
    gp.quicksum([neg_cs[i]*neg_inds[i].x for i in range(n_neg)]) + const - thresh(n)

<gurobi.LinExpr: -0.0027777777777777957>

# Use xyz

In [700]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
all_pos_inds = [H_XYZ[i][1] for i in range(1,n+1)]
all_neg_inds = [H_YXZ[i][1] for i in range(1,n+1)]
n_pos = len(concat(concat(all_pos_inds)))
n_neg = len(concat(concat(all_neg_inds)))
pos_coefs = [coef(i) for i in range(1,n+1)]
neg_coefs = [-c for c in pos_coefs]
pos_cs = concat([repeat(j, k) for j,k in zip(pos_coefs, [len(concat(ps)) for ps in all_pos_inds])])
neg_cs = concat([repeat(j, k) for j,k in zip(neg_coefs, [len(concat(ns)) for ns in all_neg_inds])])
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])

In [687]:
model = gp.Model()
# model.Params.LazyConstraints = 1
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
# model.Params.MIPFocus = 1
# model.Params.Heuristics = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-9
# model.Params.IntFeasTol = 1e-9
# model.Params.MarkowitzTol = 1e-4
# model.Params.NodeLimit = 10000
pos_inds = model.addVars(n_pos, vtype=GRB.BINARY, name='pos_inds')
neg_inds = model.addVars(n_neg, vtype=GRB.BINARY, name='neg_inds')
gx = model.addVar(lb=0, vtype='I', name='x')
gy = model.addVar(lb=1, vtype='I', name='y')
gz = model.addVar(lb=2, vtype='I', name='z')

In [688]:
obj = gp.quicksum([pos_cs[i]*pos_inds[i] for i in range(n_pos)]) + \
        gp.quicksum([neg_cs[i]*neg_inds[i] for i in range(n_neg)]) + const

In [689]:
model.setObjective(obj, GRB.MINIMIZE)
# model.addConstr(obj <= thresh(n))
model.addConstr(gx <= gy - 1)
model.addConstr(gy <= gz - 1)

<gurobi.Constr *Awaiting Model Update*>

In [690]:
all_pos_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_neg_inds

[[3*y <= z],
 [y <= 3*x],
 [y <= 2*x],
 [x <= -y + z],
 [7*y <= z],
 [2*y <= 3*x],
 [3*y - z <= x],
 [x <= -3*y + z],
 [3*y - z <= x, z < 3*y],
 [5*y <= 3*z, z < 3*y],
 [3*y <= 5*x],
 [2*y - z <= 2*x, y < 3*x],
 [y <= 2*x],
 [3*y - z <= 3*x],
 [y <= 7*x, 3*x < y],
 [3*x < y],
 [y <= 5*x, 2*x < z],
 [x <= -y + z],
 [y + z <= 3*x],
 [z < 2*x],
 [x <= -2*y + z, y < 2*x],
 [x <= -3*y + z],
 [y <= 6*x, 2*x < y],
 [2*x < y],
 [y <= 4*x, 3*x < z],
 [3*x <= -y + z],
 [-y + z < x],
 [3*x <= -y + 3*z, -y + z < x],
 [y + z <= 3*x],
 [2*x <= -y + 2*z, z < 3*x]]

In [691]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
sympy_to_gurobi = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=sympy_to_gurobi):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(sympy_to_gurobi_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [692]:
M = 20
EPS = 1
def add_constrs(variables, all_inds):
    for i, ind in enumerate(all_inds):
        rs = unify(ind)
        for r in rs:
            lhs = to_gp_linexpr(r.lhs)
            if r.rel_op == '>':
                model.addConstr(lhs >= EPS - M * (1 - variables[i]))
                model.addConstr(lhs <= M * variables[i])
            elif r.rel_op == '<=':
                model.addConstr(lhs <= M * (1 - variables[i]))
                # if ind = 0, at least one of the inequalities is violated
                if len(rs) == 1:
                    model.addConstr(lhs >= EPS - M * variables[i])
            else:
                raise ValueError

In [693]:
add_constrs(pos_inds, all_pos_inds)
add_constrs(neg_inds, all_neg_inds)

In [694]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 155 rows, 69 columns and 520 nonzeros
Model fingerprint: 0xdc71f4b1
Variable types: 0 continuous, 69 integer (66 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 2e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 0.1805556
Presolve removed 8 rows and 4 columns
Presolve time: 0.01s
Presolved: 147 rows, 65 columns, 488 nonzeros
Variable types: 0 continuous, 65 integer (62 binary)

Root relaxation: objective 1.403356e-01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.14034    0    2    0.18056    0.14034  22.3%    

In [695]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}, {not (not v.x)}")
    print(f"Objective value: {model.objVal}")
else:
    print("No feasible solution found.")

Optimal solution found:
pos_inds[0]: 0.0, False
pos_inds[1]: 1.0, True
pos_inds[2]: 0.0, False
pos_inds[3]: 1.0, True
pos_inds[4]: 1.0, True
pos_inds[5]: 0.0, False
pos_inds[6]: 0.0, False
pos_inds[7]: 0.0, False
pos_inds[8]: 0.0, False
pos_inds[9]: 1.0, True
pos_inds[10]: 1.0, True
pos_inds[11]: 0.0, False
pos_inds[12]: 1.0, True
pos_inds[13]: 0.0, False
pos_inds[14]: 0.0, False
pos_inds[15]: 0.0, False
pos_inds[16]: 0.0, False
pos_inds[17]: 0.0, False
pos_inds[18]: 0.0, False
pos_inds[19]: 1.0, True
pos_inds[20]: 1.0, True
pos_inds[21]: 1.0, True
pos_inds[22]: 1.0, True
pos_inds[23]: 0.0, False
pos_inds[24]: 0.0, False
pos_inds[25]: 0.0, False
pos_inds[26]: 1.0, True
pos_inds[27]: 1.0, True
pos_inds[28]: 1.0, True
pos_inds[29]: 1.0, True
pos_inds[30]: 0.0, False
pos_inds[31]: 0.0, False
pos_inds[32]: 0.0, False
pos_inds[33]: 0.0, False
pos_inds[34]: 0.0, False
pos_inds[35]: 0.0, False
neg_inds[0]: 1.0, True
neg_inds[1]: 1.0, True
neg_inds[2]: 1.0, True
neg_inds[3]: 1.0, True
neg_inds

In [696]:
# 3: 1,2,14
# 4: 0.5, 1.5, 4.5
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
        print(sum([sub(r, vals) for r in neg_rs]))
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [697]:
eval_f(3,3,5,15)

0
4
10


0.04351851851851851

In [682]:
H_XYZ[2]

(1, [[3*x <= y], [2*x <= z], [2*x <= y], [3*x <= z], [x <= -y + z]])